# Simulate the System on Initialized Experiment

In this notebook, we run a system simulation on some experiment which was *previously initialized* by the initialize_experiment notebook.
The simulation does not require any frame files, nor it requires to detect worm's head position at each frame. Instead, the algorithms below gather the relevant information (i.e. worm's head bbox at each frame) from a log file which was created when the experiment was initialized. As result, running a simulation of a whole experiments with tens of thousands of frames becomes a process of mere seconds.



In [ ]:
from wtracker.sim import *
from wtracker.sim.sim_controllers import *
from wtracker.sim.config import ExperimentConfig
from wtracker.utils.path_utils import join_paths
from wtracker.utils.gui_utils import UserPrompt
from wtracker.utils.frame_reader import FrameReader

### Configure the Simulation Parameters

In [ ]:
################################ User Input ################################

# The input folder containing the logs produced durning the initialization of the experiment (init_bboxes.csv) and the exp_config.json file
# if None, the user will be prompted to select the directory
input_folder = None

# the path to save the output logs of the selected controller
# if None, the user will be prompted to select the directory
output_folder = "logs"

############################################################################

if input_folder is None:
    input_folder = UserPrompt.open_directory("Select input directory containing the original logs of the experiment")

if output_folder is None:
    output_folder = UserPrompt.open_directory("Select output directory to save the logs of the selected controller")

# The path pointing to the original log file (init_bboxes.csv)
input_log_path = join_paths(input_folder, "init_bboxes.csv")

print(f"input folder: {input_folder}")
print(f"output folder: {output_folder}")

In [ ]:
# Load the experiment config
experiment_config = ExperimentConfig.load_json(f"{input_folder}/exp_config.json")

################################ User Input ################################

# initialize the timing configuration
# changing this config changes the simulation settings, should remain the same
# between different experiments
time_config = TimingConfig(
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.32, 0.32),
    experiment_config=experiment_config,
)

############################################################################

log_config = LogConfig(
    root_folder=output_folder,
    save_mic_view=False,
    save_cam_view=False,
    save_err_view=False,
    save_wrm_view=False,
)

### Choose a Controller
Run the cell of the controller you need

**MLP Controller** - Neural Network based location predictor.

In [ ]:
import torch
from wtracker.neural.mlp import WormPredictor

################################ User Input ################################

# path to the MLP model file.
# if None, the user will be prompted to select the file
model_path = None

############################################################################

if model_path is None:
    model_path = UserPrompt.open_file("Select the model file", [("pytorch files", "*.pt")])

# load the model and create the controller
model: WormPredictor = torch.load(model_path)
controller = MLPController(time_config, input_log_path, model)

**Polyfit Controller** - Weighted polynomial fitting location predictor.
The motion of the worm is fitted with a polynomial, which is afterwards used to predict the future location of the worm.

In [ ]:
## load configuration file and create the controller
# to learn more about the polyfit configuration file, see polyfit_optimizer.ipynb notebook.
poly_config = PolyfitConfig.load_json()

controller = PolyfitController(time_config, input_log_path)

**Optimal Controller** - Approximation of the absolute best controller possible.

In [ ]:
controller = OptimalController(time_config, input_log_path)

**CSV Controller** - Baseline controller without any future predictions.

In [ ]:
controller = CsvController(time_config, input_log_path)

**If you want to log the simulation run the cell below as well**  
It is recommended to run this cell, in order to be able to analyze the log and the results of the selected controller.

In [ ]:
controller = LoggingController(controller, log_config)

### Run The Simulation

In [ ]:
# create motor controller which controls the motion of the platform 
motor_controller = SineMotorController(time_config)

# create the simulator
sim = Simulator(
    time_config,
    experiment_config,
    controller,
    reader=None,
    motor_controller=motor_controller,
)

In [ ]:
# run simulation
sim.run(visualize=False, wait_key=False)

In [ ]:
# save config files
experiment_config.save_json(join_paths(output_folder, "exp_config.json"))
log_config.save_json(join_paths(output_folder, "log_config.json"))
time_config.save_json(join_paths(output_folder, "time_config.json"))

### Display some simulation-log results

In case that logging was not enabled for the simulation, the below cells are not relevant.  
For more in-depth analysis of the log results, run the plot.ipynb notebook

In [ ]:
from wtracker.eval.analysis import Plotter

# create and initialize the plotter
pltr = Plotter(
    time_config=time_config,
    log_paths=[log_config.bbox_file_path],
    plot_height=7,
)

################################ User Input ################################

pltr.initialize(unit="sec", n=10, imaging_only=True)

############################################################################

In [ ]:
pltr.print_stats()

In [ ]:
pltr.plot_speed_vs_error()

In [ ]:
pltr.plot_deviation()